In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
import functools
import pandas as pd
import re

from IPython.display import display

from cryptic.evaluation.evaluations import GeorgeHoQA
from cryptic.models.oai import OpenAIQA
from cryptic.models.selectors import SelfConsistencySelector
from cryptic.models.validators import SubWordConsistencyValidator
from cryptic.prompts.interface import PromptInterface

In [3]:
CSV_FILENAME = "data/examples/qc_1711.csv"
DUMMY_CLUE = "A farmer's son"
NUM_CLUES = None
NUM_ANSWERS = 10

Check that injection of clue and num letters into prompt template is working correctly:

In [4]:
prompt_interface = PromptInterface("cryptic.prompts.cot.ben_v1")
prompt = prompt_interface.inject_prompt(DUMMY_CLUE, 5)
print(prompt)

I will answer some clues:
    1. Boy captures the work of Shakespeare, for example (7)
    Answer: SONNETS (examples of works of Shakespeare) - SON (boy) + NETS (captures)
    2. Country folk appearing in operatic piece (7)
    Answer: ARMENIA (country) - ARIA (operatic piece) + MEN (folk)
    3. Where men are on board ship beside revolutionary (5)
    Answer: CHESS (where men are on board) - CHE (Revolutionary, Che Guevara) + SS (designation for a ship)
    4. Amputate limbs to remove weapons (6)
    Answer: DISARM (Remove weapons  - DIS-ARM (amputating limbs)
    5. Accuse one politician apiece (7)
    Answer: IMPEACH (accuse) - I (one) + MP (politician) + EACH (apiece)
    6. Orchestra was taken aback by cutting machine (7)
    Answer: BANDSAW (cutting machine) - BAND (orchestra) + SAW (was, 'taken aback' i.e. backwards)
    7. Concerning drama in second game after draw (6)
    Answer: REPLAY (a second game, typically following a drawn result in the first) - RE (meaning concerning) 

Check extraction of answer is working correctly

In [5]:
out = prompt_interface.prompt_templates["example_output"]

In [6]:
res = prompt_interface.extract_answer(out)
res

{'answer': 'REPLAY',
 'predicted_definition': 'a second game, typically following a drawn result in the first',
 'wordplay': 'RE (meaning concerning) + PLAY (a type of drama)'}

In [7]:
prompt_interface.decompose(res["wordplay"])

[{'answer': 'RE', 'definition': 'meaning concerning'},
 {'answer': 'PLAY', 'definition': 'a type of drama'}]

In [8]:
validator = SubWordConsistencyValidator(prompt_interface)

In [9]:
evaluation = GeorgeHoQA(CSV_FILENAME, num_answers=NUM_ANSWERS, num_clues=NUM_CLUES)

In [10]:
model = OpenAIQA(
    prompt_interface=prompt_interface,
    model_name="text-davinci-003",
    max_tokens=64,
    validators=[validator],
    validator_names=["consistent_decomposition"]
)

In [11]:
df = evaluation.qa_frame.df
df.head(3)

,rowid,clue,answer,definition,clue_number,puzzle_date,puzzle_name,source_url,source,num_letters
0,88,Intrepid daughter missing parent's sister (9),DAUNTLESS,Intrepid,1a,2020-09-29,Times Quick Cryptic 1711,https://times-xwd-times.livejournal.com/241434...,times_xwd_times,9
2,90,Training device transforming Liam's tour (9),SIMULATOR,Training device,8a,2020-09-29,Times Quick Cryptic 1711,https://times-xwd-times.livejournal.com/241434...,times_xwd_times,9
3,91,Plant serving sauce with meat (5),OXLIP,Plant,9a,2020-09-29,Times Quick Cryptic 1711,https://times-xwd-times.livejournal.com/241434...,times_xwd_times,5


First feed a single clue through the API to look at model output and check that we are correctly parsing responses.

This might need to be tweaked: e.g. removal of newline / space chars etc,
to convert into a standard answer format

In [12]:
clue, num_letters = evaluation.qa_frame.sample(1).df.iloc[0][["clue", "num_letters"]]

In [13]:
response = model.get_response(clue, num_letters, num_answers=2)
response

<OpenAIObject text_completion id=cmpl-7597VdRUD8sG9QibuUtflmNtYbSVc at 0x10a8d4860> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "Answer: SALAD (plant) - SAUCE (sauce) + L (meat, Lamb)"
    },
    {
      "finish_reason": "stop",
      "index": 1,
      "logprobs": null,
      "text": "\nAnswer: CHILI (plant) - LI (sauce) + CH (meat)"
    }
  ],
  "created": 1681461053,
  "id": "cmpl-7597VdRUD8sG9QibuUtflmNtYbSVc",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 44,
    "prompt_tokens": 355,
    "total_tokens": 399
  }
}

In [14]:
out = response["choices"][0]["text"]

In [15]:
res = prompt_interface.extract_answer(response["choices"][0]["text"])
res

{'answer': 'SALAD',
 'predicted_definition': 'plant',
 'wordplay': 'SAUCE (sauce) + L (meat, Lamb)'}

In [16]:
prompt_interface.decompose(res["wordplay"])

[{'answer': 'SAUCE', 'definition': 'sauce'},
 {'answer': 'L', 'definition': 'meat, Lamb'}]

In [17]:
validator.validate(res["answer"], res["predicted_definition"], res["wordplay"])

False

Once we're confident that answers are being extracted correctly we can run a whole set of clues through

In [18]:
metrics, answer_df = evaluation.run(model)

23it [01:28,  3.84s/it]


In [19]:
model.validator_names

['consistent_decomposition']

In [20]:
pred_cols = ["prediction", "predicted_definition", "wordplay"]
vis_cols = pred_cols + model.validator_names + ["prediction_satisfies_constraints"]

In [21]:
for i in range(NUM_ANSWERS):
    display(answer_df[["clue", "answer"] + [c + f"-{i}" for c in vis_cols]])

,clue,answer,prediction-0,predicted_definition-0,wordplay-0,consistent_decomposition-0,prediction_satisfies_constraints-0
0,Intrepid daughter missing parent's sister (9),DAUNTLESS,BRAVERIES,intrepid,BRAVE (daughter) + RIES (parent's sister),True,True
2,Training device transforming Liam's tour (9),SIMULATOR,EXERCISER,training device,"EX (transforming) + CIRES (Liam's tour, anagra...",False,True
3,Plant serving sauce with meat (5),OXLIP,GRAVY,sauce served with meat,GRA (plant) + VY (serving),True,True
4,Clothing submerged in northern river? (9),UNDERWEAR,OVERCOAT,clothing,OVER (submerged) + CO (Northern) + AT (river),True,False
5,"Popular act, without doubt (6)",INDEED,CROWD,popular act,DOUBT (without doubt),False,False
6,Position of rubbish in South America (6),STATUS,GARBAGE,position of rubbish,GAR (South America) + BAGE (in),True,False
7,Element Mendeleev originally found in flowerin...,GERMANIUM,CARBONIUM,element Mendeleev originally found,CAR (flowering plant) + BONIUM (originally),True,True
8,"Head of police called, revealing accident (5)",PRANG,EMERGED,revealing accident,"EME (head of police, Chief Inspector) + RGED (...",True,False
9,Switch posts near ground (9),TRANSPOSE,EXCHANGE,switch,EX (posts) + CHANGE (near ground),True,False
10,"With cunning, son locates youth leaders twice (5)",SLYLY,SNEAK,with cunning,S (son) + NE (locates) + A (youth) + K (leader...,True,True


,clue,answer,prediction-1,predicted_definition-1,wordplay-1,consistent_decomposition-1,prediction_satisfies_constraints-1
0,Intrepid daughter missing parent's sister (9),DAUNTLESS,ADVENTURESS,intrepid daughter,AD (parent) + VENTURE (missing) + SS (sister),True,False
2,Training device transforming Liam's tour (9),SIMULATOR,SIMULATOR,training device,"SIMUL (transform) + ATOR (Liam's tour, i.e. Li...",True,True
3,Plant serving sauce with meat (5),OXLIP,GRAVY,sauce with meat,GR (plant) + AY (serving),False,True
4,Clothing submerged in northern river? (9),UNDERWEAR,OVERCOAT,clothing,"OVER (submerged) + CO (Northern river, the Coq...",False,False
5,"Popular act, without doubt (6)",INDEED,HITMEN,popular act,HINT (doubt) + MEN (without),False,True
6,Position of rubbish in South America (6),STATUS,TRASHIN,position of rubbish,TR (South America abbreviation) + ASHIN (in),True,False
7,Element Mendeleev originally found in flowerin...,GERMANIUM,CARBONATE,element Mendeleev originally found,CARBON (element) + ATE (flowering plant),True,True
8,"Head of police called, revealing accident (5)",PRANG,,,,True,False
9,Switch posts near ground (9),TRANSPOSE,EXCHANGE,switch,EX (posts) + CHANGE (near ground),True,False
10,"With cunning, son locates youth leaders twice (5)",SLYLY,,,,True,False


,clue,answer,prediction-2,predicted_definition-2,wordplay-2,consistent_decomposition-2,prediction_satisfies_constraints-2
0,Intrepid daughter missing parent's sister (9),DAUNTLESS,ADVENTURESS,intrepid daughter,AD (parent's sister) + VENTURE (missing) + S (...,False,False
2,Training device transforming Liam's tour (9),SIMULATOR,EXERCISER,training device,EX (transforming) + ER (Liam's) + CISE (tour),False,True
3,Plant serving sauce with meat (5),OXLIP,HERBS,plant,HERB (sauce) + S (meat),True,True
4,Clothing submerged in northern river? (9),UNDERWEAR,OVERCOAT,clothing,"OVER (submerged) + COAT (northern river, River...",True,False
5,"Popular act, without doubt (6)",INDEED,CROWDED,popular act,CERTAIN (without doubt) + D (doubt),False,False
6,Position of rubbish in South America (6),STATUS,TRASHAN,position of rubbish,TRASH (rubbish) + AN (South America),True,False
7,Element Mendeleev originally found in flowerin...,GERMANIUM,CARBONIUM,element,CAR (Mendeleev's symbol for Carbon) + BONIUM (...,True,True
8,"Head of police called, revealing accident (5)",PRANG,UNCAP,revealing accident,UN (head) + CAP (police called),True,True
9,Switch posts near ground (9),TRANSPOSE,EXCHANGE,switch posts,EX (near) + CHANGE (ground),True,False
10,"With cunning, son locates youth leaders twice (5)",SLYLY,SNEAK,with cunning,SN (son) + EAK (locate twice),True,True


,clue,answer,prediction-3,predicted_definition-3,wordplay-3,consistent_decomposition-3,prediction_satisfies_constraints-3
0,Intrepid daughter missing parent's sister (9),DAUNTLESS,ADVENTURER,intrepid,AD (daughter) + VENT (parent's sister) + UR (m...,False,False
2,Training device transforming Liam's tour (9),SIMULATOR,EXERCISER,training device,"EX (tour, expedition) + CRIER (Liam's, as in L...",False,True
3,Plant serving sauce with meat (5),OXLIP,SALAD,plant,SAUCE (sauce) + LAD (meat),False,True
4,Clothing submerged in northern river? (9),UNDERWEAR,OVERCOAT,clothing,OVER (submerged) + COAT (river in northern Eng...,True,False
5,"Popular act, without doubt (6)",INDEED,MUSTER,popular act,MU (without) + STER (doubt),True,True
6,Position of rubbish in South America (6),STATUS,TRASHEN,position of rubbish,"TRASH (rubbish) + EN (South America, abbreviat...",True,False
7,Element Mendeleev originally found in flowerin...,GERMANIUM,CARBONATE,element Mendeleev originally found,CARBON (element) + ATE (flowering plant),True,True
8,"Head of police called, revealing accident (5)",PRANG,,,,True,False
9,Switch posts near ground (9),TRANSPOSE,EXCHANGE,switch posts,EX (near) + CHANGE (ground),True,False
10,"With cunning, son locates youth leaders twice (5)",SLYLY,,,,True,False


,clue,answer,prediction-4,predicted_definition-4,wordplay-4,consistent_decomposition-4,prediction_satisfies_constraints-4
0,Intrepid daughter missing parent's sister (9),DAUNTLESS,ADVENTURE,intrepid,AD (daughter) + VENTURE (parent's sister),True,True
2,Training device transforming Liam's tour (9),SIMULATOR,EXERCISER,training device,"EX (Liam's tour, as in ""Ex tour"") + CISE (tran...",False,True
3,Plant serving sauce with meat (5),OXLIP,STEWS,plant serving sauce,S (with) + TEW (meat),False,True
4,Clothing submerged in northern river? (9),UNDERWEAR,OVERCOAT,clothing,"OVER (submerged) + COAT (river, in Northern En...",True,False
5,"Popular act, without doubt (6)",INDEED,CROWDED,popular act,CROWD (without doubt) + ED (abbreviation for '...,True,False
6,Position of rubbish in South America (6),STATUS,,,,True,False
7,Element Mendeleev originally found in flowerin...,GERMANIUM,CARBONIUM,element Mendeleev originally found,CAR (flowering plant) + BONIUM (originally),True,True
8,"Head of police called, revealing accident (5)",PRANG,,,,True,False
9,Switch posts near ground (9),TRANSPOSE,EXCHANGE,switch posts,EX (near) + CHANGE (ground),True,False
10,"With cunning, son locates youth leaders twice (5)",SLYLY,SNEAK,with cunning,"S (son) + NE (locates) + AK (youth leaders, AK...",False,True


,clue,answer,prediction-5,predicted_definition-5,wordplay-5,consistent_decomposition-5,prediction_satisfies_constraints-5
0,Intrepid daughter missing parent's sister (9),DAUNTLESS,ADVENTURE,intrepid,AUNT (parent's sister) + VENTURE (daughter),False,True
2,Training device transforming Liam's tour (9),SIMULATOR,EXERCISER,training device,EX (transform) + CRUISE (Liam's tour),False,True
3,Plant serving sauce with meat (5),OXLIP,STEWS,plant serving sauce,S (sauce) + TEW (meat),False,True
4,Clothing submerged in northern river? (9),UNDERWEAR,OVERCOAT,clothing,OVER (submerged) + COAT (river in Northern Eng...,True,False
5,"Popular act, without doubt (6)",INDEED,TRENDY,popular act,TRY (without doubt) + ND (doubt),False,True
6,Position of rubbish in South America (6),STATUS,TRASHEN,position of rubbish,"TRASH (rubbish) + EN (South America, abbreviat...",True,False
7,Element Mendeleev originally found in flowerin...,GERMANIUM,CARBONIUM,element Mendeleev originally found,CAR (flowering plant) + BONIUM (originally),True,True
8,"Head of police called, revealing accident (5)",PRANG,,,,True,False
9,Switch posts near ground (9),TRANSPOSE,EXCHANGE,switch,EX (posts) + CHANGE (near ground),True,False
10,"With cunning, son locates youth leaders twice (5)",SLYLY,,,,True,False


,clue,answer,prediction-6,predicted_definition-6,wordplay-6,consistent_decomposition-6,prediction_satisfies_constraints-6
0,Intrepid daughter missing parent's sister (9),DAUNTLESS,ADVENTURE,intrepid,AD (daughter) + VENT (parent's sister) + URE (...,True,True
2,Training device transforming Liam's tour (9),SIMULATOR,EXERCISER,training device,"EX (Liam's tour, e.g. Liam's Expedition) + RCI...",False,True
3,Plant serving sauce with meat (5),OXLIP,MASHED,sauce with meat,MASH (plant) + ED (serving),True,False
4,Clothing submerged in northern river? (9),UNDERWEAR,OVERCOAT,clothing,"OVER (submerged) + COAT (river, as in the Rive...",True,False
5,"Popular act, without doubt (6)",INDEED,CROWDS,popular act,C (without) + ROWD (doubt),False,True
6,Position of rubbish in South America (6),STATUS,TRASHEN,position of rubbish,"TRASH (rubbish) + EN (South America, short for...",True,False
7,Element Mendeleev originally found in flowerin...,GERMANIUM,CARBONATE,element Mendeleev originally found,CARBON (found in flowering plant) + ATE (Mende...,True,True
8,"Head of police called, revealing accident (5)",PRANG,UNCOIL,revealing accident,"UN (head) + CO (police, short for constable) +...",True,False
9,Switch posts near ground (9),TRANSPOSE,EXCHANGE,switch,EX (posts) + CHANGE (near ground),True,False
10,"With cunning, son locates youth leaders twice (5)",SLYLY,,,,True,False


,clue,answer,prediction-7,predicted_definition-7,wordplay-7,consistent_decomposition-7,prediction_satisfies_constraints-7
0,Intrepid daughter missing parent's sister (9),DAUNTLESS,ADVENTURER,intrepid,AD (daughter) + VENTURE (missing) + R (parent'...,True,False
2,Training device transforming Liam's tour (9),SIMULATOR,EXERCISER,training device,EX (transforming) + CIR (tour) + ES (Liam's),False,True
3,Plant serving sauce with meat (5),OXLIP,SALSA,plant serving sauce,"SAUCE (sauce) + L (meat, Lamb)",False,True
4,Clothing submerged in northern river? (9),UNDERWEAR,OVERCOAT,clothing,OVER (submerged) + COAT (river in Northern Eng...,True,False
5,"Popular act, without doubt (6)",INDEED,CROWDED,popular act,C (without) + ROWDED (doubt),True,False
6,Position of rubbish in South America (6),STATUS,TRASHES,position of rubbish,"TR (South America, short for 'Tierra del Fuego...",True,False
7,Element Mendeleev originally found in flowerin...,GERMANIUM,CARBONATE,element Mendeleev originally found,CARBON (element) + ATE (flowering plant),True,True
8,"Head of police called, revealing accident (5)",PRANG,EMERGED,revealing accident,EME (head of police) + RGED (called),True,False
9,Switch posts near ground (9),TRANSPOSE,EXCHANGE,switch,EX (near) + CHANGE (ground),True,False
10,"With cunning, son locates youth leaders twice (5)",SLYLY,,,,True,False


,clue,answer,prediction-8,predicted_definition-8,wordplay-8,consistent_decomposition-8,prediction_satisfies_constraints-8
0,Intrepid daughter missing parent's sister (9),DAUNTLESS,DAUNTLESS,intrepid,DAU (daughter) + NT (missing parent's) + LESS ...,True,True
2,Training device transforming Liam's tour (9),SIMULATOR,EXERCISER,training device,"EX (Liam's tour, 'Ex on the Beach') + CISE (tr...",False,True
3,Plant serving sauce with meat (5),OXLIP,HERBS,plant,HER (serving) + BS (sauce) + MEAT (meat),False,True
4,Clothing submerged in northern river? (9),UNDERWEAR,OVERCOAT,clothing,OVER (submerged) + COAT (river in Northern Eng...,True,False
5,"Popular act, without doubt (6)",INDEED,CROWDED,popular act,CROWD (without) + ED (doubt),True,False
6,Position of rubbish in South America (6),STATUS,TRASHES,position of rubbish,TR (abbreviation for 'south' in South America)...,True,False
7,Element Mendeleev originally found in flowerin...,GERMANIUM,CARBONIUM,element Mendeleev originally found,CAR (flowering plant) + BONIUM (originally),True,True
8,"Head of police called, revealing accident (5)",PRANG,,,,True,False
9,Switch posts near ground (9),TRANSPOSE,EXCHANGE,switch,EX (posts) + CHANGE (near ground),True,False
10,"With cunning, son locates youth leaders twice (5)",SLYLY,,,,True,False


,clue,answer,prediction-9,predicted_definition-9,wordplay-9,consistent_decomposition-9,prediction_satisfies_constraints-9
0,Intrepid daughter missing parent's sister (9),DAUNTLESS,ADVENTURER,intrepid,AD (daughter) + VENTURE (parent's sister),False,False
2,Training device transforming Liam's tour (9),SIMULATOR,EXERCISER,training device,EX (transform) + CIRCLE (tour) + S (Liam's),False,True
3,Plant serving sauce with meat (5),OXLIP,HERBS,plant,HER (serving) + BS (sauce),True,True
4,Clothing submerged in northern river? (9),UNDERWEAR,OVERCOAT,clothing,OVER (submerged) + COAT (river in Northern Eng...,True,False
5,"Popular act, without doubt (6)",INDEED,TRENDY,popular act,TRY (doubt) + ND (without),False,True
6,Position of rubbish in South America (6),STATUS,,,,True,False
7,Element Mendeleev originally found in flowerin...,GERMANIUM,CARBONIUM,element Mendeleev originally found,CAR (flowering plant) + BONIUM (originally),True,True
8,"Head of police called, revealing accident (5)",PRANG,EMERGED,revealing accident,"EME (head of police, Chief Inspector) + RGED (...",True,False
9,Switch posts near ground (9),TRANSPOSE,EXCHANGE,switch,EX (posts) + CHANGE (near ground),True,False
10,"With cunning, son locates youth leaders twice (5)",SLYLY,,,,True,False


I want to somehow use the validator columns to prioritise answers.

I think it might be useful to push the dataframe to long form.

In [25]:
long_df = pd.wide_to_long(answer_df, vis_cols, i="rowid", j="index", sep="-")

In [26]:
long_df

,,correct,answer,clue,puzzle_name,source_url,definition,source,num_letters,clue_number,puzzle_date,prediction,predicted_definition,wordplay,consistent_decomposition,prediction_satisfies_constraints
rowid,index,,,,,,,,,,,,,,,
88,0,True,DAUNTLESS,Intrepid daughter missing parent's sister (9),Times Quick Cryptic 1711,https://times-xwd-times.livejournal.com/241434...,Intrepid,times_xwd_times,9,1a,2020-09-29,BRAVERIES,intrepid,BRAVE (daughter) + RIES (parent's sister),True,True
90,0,True,SIMULATOR,Training device transforming Liam's tour (9),Times Quick Cryptic 1711,https://times-xwd-times.livejournal.com/241434...,Training device,times_xwd_times,9,8a,2020-09-29,EXERCISER,training device,"EX (transforming) + CIRES (Liam's tour, anagra...",False,True
91,0,False,OXLIP,Plant serving sauce with meat (5),Times Quick Cryptic 1711,https://times-xwd-times.livejournal.com/241434...,Plant,times_xwd_times,5,9a,2020-09-29,GRAVY,sauce served with meat,GRA (plant) + VY (serving),True,True
92,0,False,UNDERWEAR,Clothing submerged in northern river? (9),Times Quick Cryptic 1711,https://times-xwd-times.livejournal.com/241434...,Clothing,times_xwd_times,9,10a,2020-09-29,OVERCOAT,clothing,OVER (submerged) + CO (Northern) + AT (river),True,False
93,0,False,INDEED,"Popular act, without doubt (6)",Times Quick Cryptic 1711,https://times-xwd-times.livejournal.com/241434...,without doubt,times_xwd_times,6,12a,2020-09-29,CROWD,popular act,DOUBT (without doubt),False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,9,True,ENTREATY,Plea to dine among competitors (8),Times Quick Cryptic 1711,https://times-xwd-times.livejournal.com/241434...,Plea,times_xwd_times,8,11d,2020-09-29,ENTERTAIN,plea to dine,EN (among) + TER (competitors) + TAIN (competi...,True,False
108,9,False,TEMPEST,Storm created by English politician during tri...,Times Quick Cryptic 1711,https://times-xwd-times.livejournal.com/241434...,Storm,times_xwd_times,7,14d,2020-09-29,,,,True,False
109,9,True,MIMOSA,"Shrub found in Michigan, Missouri and South Am...",Times Quick Cryptic 1711,https://times-xwd-times.livejournal.com/241434...,Shrub,times_xwd_times,6,15d,2020-09-29,BUSHEL,shrub,"BUSH (found in Michigan, Missouri) + EL (South...",True,True


In [27]:
pred_cols = ["prediction", "predicted_definition", "wordplay"] + model.validator_names + ["prediction_satisfies_constraints"]

In [28]:
selector = SelfConsistencySelector(model.validator_names)

In [30]:
selected = selector.select(answer_df)

In [37]:
len(answer_df), answer_df["correct"].sum()

(23, 8)

In [41]:
len(selected), selected["correct"].sum()

(16, 7)